In [1]:
# Importando as bibliotecas utilizadas
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [18]:
# Criando a sessão, nomeando e configurando para mostrar tabela do tipo pandas 
spark = (
    SparkSession.builder
    .appName('PySpark - Join')
    .config('spark.sql.repl.eagerEval.enabled', True)
    .getOrCreate()
)

In [19]:
# Carregando as bases de dados em PARQUET para trabalhar com o join
autores = spark.read.parquet('./DATASETS/AUTORES.parquet')
livros = spark.read.parquet('./DATASETS/LIVROS.parquet')
compras = spark.read.parquet('./DATASETS/COMPRAS.parquet')
clientes = spark.read.parquet('./DATASETS/CLIENTES.parquet')

In [4]:
autores

id,titulo,autor
106042959,1984,George Orwell
017551704,200 Crônicas Esco...,Rubem Braga
054099410,A Alma Encantador...,João do Rio
128422117,A Biblioteca da M...,Matt Haig
049141399,A Coleira do Cão,Rubem Fonseca
065490862,A Cor Púrpura,Alice Walker
029077001,A Educação Sentim...,Gustave Flaubert
032275599,A Escrava Isaura,Bernardo Guimarães
090690941,A Estrela Sobe,Marques Rebelo
095695255,A Hora da Estrela,Clarice Lispector


In [5]:
livros

id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
102961160,54.317.982/0001-79,2011-09-19,4367115749184,0-06-594558-1,516,113.84
043636240,16.938.042/0001-08,2019-05-11,1239569754256,0-611-23680-X,371,207.67
052928059,57.624.038/0001-52,2003-08-03,5462162528221,1-109-29542-1,198,107.89
088482271,16.249.378/0001-63,2022-09-07,5713629047213,1-229-68842-0,632,143.95
028125446,90.425.178/0001-77,2011-07-25,2226757040245,0-657-66391-3,384,156.49
055610005,20.756.194/0001-67,2006-03-22,2114468195241,0-455-43278-3,419,37.08
049806867,93.617.240/0001-10,2022-02-17,3166123637398,1-4822-8784-6,378,162.97
102803370,48.760.531/0001-09,2011-12-13,2550608407612,0-399-05121-X,732,75.62
084858867,10.752.843/0001-06,2015-04-30,6207011925788,1-03-549652-6,635,237.67
059303729,05.129.783/0001-21,2001-08-25,1641133648784,0-620-46508-5,107,61.41


In [6]:
clientes

id,cpf,rg,name,email,data_de_nascimento,estado,profissao,telefone
4738995,637.180.254-26,012835766,Srta. Ana Laura M...,eteixeira@uol.com.br,2003-02-03,RS,Profissional De R...,(011) 8424-0376
4011280,689.750.231-68,01284357X,Ana Julia das Neves,xda-paz@hotmail.com,2009-05-29,PA,Engenheiro Sanita...,+55 71 9739-6705
5662332,306.158.472-26,013562745,Milena Cunha,pietrada-rosa@bol...,2013-10-05,AC,Fonoaudiólogo,+55 81 5498-3331
7450009,593.140.268-33,014736822,Sra. Eloah Nogueira,yda-paz@bol.com.br,1979-11-02,PI,Profissional De T...,(021) 8628-7265
3583249,982.604.317-69,014876358,Nathan Azevedo,rafaelacostela@ya...,1931-08-29,ES,Profissional De C...,81 2925 1456
2890701,721.395.064-99,01528637X,Esther Rodrigues,caue34@yahoo.com.br,1957-01-16,RO,Senador,(011) 5381 6173
6685012,867.439.052-83,015842733,João Gabriel Barros,pedro-lucas13@gma...,1982-06-17,AM,Neonatologista,(071) 5210 2730
2388467,470.815.269-85,01625837X,Luiz Otávio Rezende,portojoana@uol.co...,2017-07-16,MS,Profissional De L...,+55 71 5952-3387
4279570,857.210.643-07,016487357,Gustavo Henrique ...,da-matabenjamin@i...,1983-06-24,BA,Engenheiro Agrícola,31 6727 6524
8599757,810.563.427-90,017346587,Pietro Nunes,carvalholeandro@h...,2012-09-03,PI,Cobaia Médica,+55 (061) 3135-5607


In [7]:
compras

id,cartao_data_expiracao,cartao_numero,cartao_bandeira,cartao_cvc,codigo_transacao_bancaria,data,hora,ipv4,ipv6,cep_entrega,cd_livro,cd_cliente
012389,11/25,5500804500517692,Discover,959,GB98MPIH622108593...,2021-07-24,03:21:28,62.145.31.164,b23d:58a2:9eff:36...,36629-219,030334762,3339828
012476,01/31,4609489235873,VISA 16 digit,6979,GB79GAVL233010819...,2021-08-23,11:15:52,185.150.224.52,1e2c:f641:49d1:5a...,17012-747,013721981,7624624
012478,07/23,30072722359174,American Express,689,GB51SZOU538484531...,2020-01-27,17:08:10,40.179.153.24,9bcd:ee0c:af68:33...,25921298,019458805,8703114
012534,09/29,4561935154572,VISA 16 digit,353,GB05UNEX021466511...,2021-07-15,07:39:25,86.20.51.194,be57:8f94:132:1cb...,73799370,014347542,7799936
012549,01/31,3541220668415122,JCB 15 digit,967,GB97RCCC581942620...,2020-07-22,23:56:14,55.189.220.65,a1fd:9dd:115f:9d5...,50727-454,020215846,6703678
012574,06/24,4155061214506542,JCB 16 digit,362,GB93YATI197164292...,2020-05-19,10:47:09,212.26.253.42,1815:dc0e:c557:96...,67662-182,010325500,6273720
012579,01/27,30554213514227,Maestro,9847,GB54PQLN799705859...,2021-04-15,10:36:56,168.197.230.167,7f3b:4ada:4c62:3f...,61193-110,053479015,6977964
012648,05/24,347386847428278,VISA 16 digit,823,GB90MYZL510193283...,2021-06-19,13:07:48,146.108.131.11,c52d:53d8:371a:34...,10059-723,012331534,6616715
012675,04/29,4534805013764,Mastercard,384,GB11XMUH824876351...,2021-07-11,07:55:08,148.155.72.234,b290:d2b7:6a58:86...,79292884,035940339,649001
012698,12/31,4450657280386776,American Express,247,GB79ECRO017868447...,2021-08-11,12:06:48,190.135.5.172,7cbc:4554:7f5d:65...,93609312,058320651,4523531


In [8]:
# Fazendo left join entre os datasets autores e livros
autores.join(other=livros, on='id', how='left')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
106042959,1984,George Orwell,NULL,NULL,NULL,NULL,NULL,NULL
017551704,200 Crônicas Esco...,Rubem Braga,14.857.032/0001-12,2018-10-19,2352660010751,0-7433-6026-5,645,44.38
054099410,A Alma Encantador...,João do Rio,NULL,NULL,NULL,NULL,NULL,NULL
128422117,A Biblioteca da M...,Matt Haig,NULL,NULL,NULL,NULL,NULL,NULL
049141399,A Coleira do Cão,Rubem Fonseca,NULL,NULL,NULL,NULL,NULL,NULL
065490862,A Cor Púrpura,Alice Walker,NULL,NULL,NULL,NULL,NULL,NULL
029077001,A Educação Sentim...,Gustave Flaubert,NULL,NULL,NULL,NULL,NULL,NULL
032275599,A Escrava Isaura,Bernardo Guimarães,49.037.512/0001-11,2001-05-15,2964406383663,1-83335-705-1,752,184.42
090690941,A Estrela Sobe,Marques Rebelo,NULL,NULL,NULL,NULL,NULL,NULL
095695255,A Hora da Estrela,Clarice Lispector,94.567.038/0001-94,2015-04-23,4084520465552,0-7650-7381-1,572,83.33


In [9]:
# Fazendo left right entre os datasets autores e livros
autores.join(other=livros, on='id', how='right')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
102961160,NULL,NULL,54.317.982/0001-79,2011-09-19,4367115749184,0-06-594558-1,516,113.84
043636240,NULL,NULL,16.938.042/0001-08,2019-05-11,1239569754256,0-611-23680-X,371,207.67
052928059,NULL,NULL,57.624.038/0001-52,2003-08-03,5462162528221,1-109-29542-1,198,107.89
088482271,NULL,NULL,16.249.378/0001-63,2022-09-07,5713629047213,1-229-68842-0,632,143.95
028125446,NULL,NULL,90.425.178/0001-77,2011-07-25,2226757040245,0-657-66391-3,384,156.49
055610005,NULL,NULL,20.756.194/0001-67,2006-03-22,2114468195241,0-455-43278-3,419,37.08
049806867,NULL,NULL,93.617.240/0001-10,2022-02-17,3166123637398,1-4822-8784-6,378,162.97
102803370,NULL,NULL,48.760.531/0001-09,2011-12-13,2550608407612,0-399-05121-X,732,75.62
084858867,NULL,NULL,10.752.843/0001-06,2015-04-30,6207011925788,1-03-549652-6,635,237.67
059303729,NULL,NULL,05.129.783/0001-21,2001-08-25,1641133648784,0-620-46508-5,107,61.41


In [10]:
# Fazendo full join entre os datasets autores e livros
autores.join(other=livros, on='id', how='full')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
010033025,NULL,NULL,06.849.371/0001-29,2005-07-05,3363492241854,1-196-97629-5,095,80.63
010045097,Moby Dick,Herman Melville,93.167.508/0001-60,2003-12-17,5404258649298,1-198-80360-6,209,138.69
010249547,NULL,NULL,10.345.968/0001-03,2017-12-04,7005400578046,0-7705-1411-1,246,81.2
010325500,A Obscena Senhora D,Hilda Hilst,27.689.540/0001-80,2000-09-09,8793472196570,0-933627-98-X,306,30.62
010366161,Lucíola,José de Alencar,70.293.541/0001-30,2022-02-18,9429320742345,0-356-75493-6,968,200.6
010409138,Romanceiro da Inc...,Cecília Meireles,NULL,NULL,NULL,NULL,NULL,NULL
010745876,Gargântua e Panta...,François Rabelais,01.427.369/0001-48,2017-06-12,0666233842154,0-14-758390-X,910,237.77
010838220,Ficções,Jorge Luis Borges,34.927.681/0001-50,2009-06-15,7773545068127,0-536-43094-2,368,129.9
011195894,Journey to the En...,Louis-Ferdinand C...,NULL,NULL,NULL,NULL,NULL,NULL
011247991,Lavoura Arcaica,Raduan Nassar,97.352.048/0001-09,2009-05-03,1126815895613,0-625-59780-X,616,106.29


In [11]:
# Fazendo inner join entre os datasets autores e livros
autores.join(other=livros, on='id', how='inner')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
080608956,Viva o Povo Brasi...,João Ubaldo Ribeiro,90.563.247/0001-09,2011-10-05,0332981484891,1-925760-25-1,203,107.63
018453254,Grande Sertão: Ve...,Guimarães Rosa,95.367.280/0001-87,2012-11-02,0507454098268,0-609-21722-4,043,52.94
021765997,As Aventuras de H...,Mark Twain,59.374.801/0001-60,2014-05-17,4743679337089,1-85580-232-5,499,143.43
023911383,Catatau,Paulo Leminski,98.526.047/0001-98,2005-11-12,4165252098109,1-369-22273-4,599,121.0
030099528,Retrato do Artist...,James Joyce,28.103.975/0001-64,2000-02-06,9164658045347,0-904222-71-3,292,208.39
030334762,Em Busca do Tempo...,Marcel Proust,80.463.719/0001-06,2013-05-08,7122725133429,0-85193-340-8,099,123.47
035940339,Adeus às Armas,Ernest Hemingway,71.530.496/0001-53,2014-08-11,6764222732753,1-396-90787-4,327,213.6
026925428,"Sing, Unburied, S...",Jesmyn Ward,97.156.423/0001-37,2011-09-04,7828100308177,0-491-15648-0,793,193.19
010045097,Moby Dick,Herman Melville,93.167.508/0001-60,2003-12-17,5404258649298,1-198-80360-6,209,138.69
010366161,Lucíola,José de Alencar,70.293.541/0001-30,2022-02-18,9429320742345,0-356-75493-6,968,200.6


In [12]:
# Fezendo join com colunas de nome diferente 
compras.join(livros, [compras.cd_livro == livros.id], 'inner')

id,cartao_data_expiracao,cartao_numero,cartao_bandeira,cartao_cvc,codigo_transacao_bancaria,data,hora,ipv4,ipv6,cep_entrega,cd_livro,cd_cliente,id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
012389,11/25,5500804500517692,Discover,959,GB98MPIH622108593...,2021-07-24,03:21:28,62.145.31.164,b23d:58a2:9eff:36...,36629-219,030334762,3339828,030334762,80.463.719/0001-06,2013-05-08,7122725133429,0-85193-340-8,099,123.47
012476,01/31,4609489235873,VISA 16 digit,6979,GB79GAVL233010819...,2021-08-23,11:15:52,185.150.224.52,1e2c:f641:49d1:5a...,17012-747,013721981,7624624,013721981,76.351.940/0001-97,2004-01-27,8786391051693,1-60705-523-6,379,133.43
012478,07/23,30072722359174,American Express,689,GB51SZOU538484531...,2020-01-27,17:08:10,40.179.153.24,9bcd:ee0c:af68:33...,25921298,019458805,8703114,019458805,81.253.490/0001-30,2007-12-13,2628486550239,0-916523-25-X,788,238.54
012534,09/29,4561935154572,VISA 16 digit,353,GB05UNEX021466511...,2021-07-15,07:39:25,86.20.51.194,be57:8f94:132:1cb...,73799370,014347542,7799936,014347542,76.415.890/0001-64,2007-04-05,3883750237349,0-8360-5203-X,452,182.43
012549,01/31,3541220668415122,JCB 15 digit,967,GB97RCCC581942620...,2020-07-22,23:56:14,55.189.220.65,a1fd:9dd:115f:9d5...,50727-454,020215846,6703678,020215846,27.659.813/0001-44,2014-03-25,7684001180416,0-216-16469-9,794,155.18
012574,06/24,4155061214506542,JCB 16 digit,362,GB93YATI197164292...,2020-05-19,10:47:09,212.26.253.42,1815:dc0e:c557:96...,67662-182,010325500,6273720,010325500,27.689.540/0001-80,2000-09-09,8793472196570,0-933627-98-X,306,30.62
012579,01/27,30554213514227,Maestro,9847,GB54PQLN799705859...,2021-04-15,10:36:56,168.197.230.167,7f3b:4ada:4c62:3f...,61193-110,053479015,6977964,053479015,54.327.896/0001-47,2018-12-21,1750535865566,0-518-04786-5,105,207.52
012648,05/24,347386847428278,VISA 16 digit,823,GB90MYZL510193283...,2021-06-19,13:07:48,146.108.131.11,c52d:53d8:371a:34...,10059-723,012331534,6616715,012331534,84.317.925/0001-50,2010-05-29,3177832263150,1-62649-755-9,129,175.67
012675,04/29,4534805013764,Mastercard,384,GB11XMUH824876351...,2021-07-11,07:55:08,148.155.72.234,b290:d2b7:6a58:86...,79292884,035940339,649001,035940339,71.530.496/0001-53,2014-08-11,6764222732753,1-396-90787-4,327,213.6
012698,12/31,4450657280386776,American Express,247,GB79ECRO017868447...,2021-08-11,12:06:48,190.135.5.172,7cbc:4554:7f5d:65...,93609312,058320651,4523531,058320651,82.439.107/0001-03,2013-01-30,6024459409867,1-61357-201-8,290,196.35


In [13]:
# Fezendo join com a condição em uma LISTA separada
condicao = [compras.cd_livro == livros.id, livros.preco < 20, compras.cartao_bandeira == 'Mastercard']
compras.join(livros,  condicao, 'inner')

id,cartao_data_expiracao,cartao_numero,cartao_bandeira,cartao_cvc,codigo_transacao_bancaria,data,hora,ipv4,ipv6,cep_entrega,cd_livro,cd_cliente,id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
016284,07/25,4995253124034962,Mastercard,316,GB45KZNR247418910...,2020-10-06,05:12:49,94.195.142.102,7ee3:bf8c:b0b1:5a...,40695-781,012719596,3713338,012719596,91.572.403/0001-60,2019-09-02,0416248524886,1-71012-481-4,617,11.74
021783,09/24,3587017635827925,Mastercard,382,GB95JQHA912163308...,2022-10-28,05:33:09,136.229.183.46,1811:d388:ac92:f3...,66885-884,028590563,4971867,028590563,83.652.071/0001-04,2017-10-01,4098655700235,0-09-656628-0,580,18.31
026184,05/27,4213460012148657,Mastercard,895,GB38JREB002871238...,2020-02-16,08:43:56,64.150.100.169,bec3:500c:759a:a2...,56539-909,097717319,3731803,097717319,29.635.084/0001-11,2018-01-27,6029658867845,0-417-50357-1,919,10.87
061498,01/31,6541688476149666,Mastercard,492,GB51SCPD648967971...,2023-01-25,02:21:09,159.245.209.149,6c21:749c:e888:14...,11376-043,020113375,4738995,020113375,72.513.640/0001-06,2004-05-20,6368864423927,1-4262-9302-X,979,11.22
063814,11/31,6571799209250581,Mastercard,631,GB25EHYY003596885...,2022-09-30,12:04:19,190.133.1.142,3adc:1cb6:9dfe:b8...,33797954,020113375,8353624,020113375,72.513.640/0001-06,2004-05-20,6368864423927,1-4262-9302-X,979,11.22
068921,12/29,4594726591421247,Mastercard,774,GB72PEKY779837453...,2023-01-21,12:32:32,21.117.238.69,b363:ca20:5a3f:97...,76711-449,022112497,3289985,022112497,82.074.936/0001-21,2021-01-06,5078165920063,0-7763-1592-7,799,10.84
076123,02/29,6011953717127074,Mastercard,697,GB55SSHK295167521...,2021-11-18,07:36:26,112.177.151.130,76a5:b04f:7410:94...,77482079,012719596,2164688,012719596,91.572.403/0001-60,2019-09-02,0416248524886,1-71012-481-4,617,11.74
076951,09/32,4158441267736617,Mastercard,353,GB61DKSF920635726...,2022-02-26,16:48:26,96.159.193.161,f8e4:afb0:b430:6c...,21991812,028590563,471687,028590563,83.652.071/0001-04,2017-10-01,4098655700235,0-09-656628-0,580,18.31
097486,03/32,502059551627,Mastercard,132,GB42SAXW397890443...,2023-03-16,08:42:36,171.163.255.248,509e:bca0:eca:b9b...,37407335,025900512,2662943,025900512,28.359.407/0001-29,2001-04-24,2793458725790,1-175-20698-9,623,12.32
107945,12/31,5585811658738465,Mastercard,658,GB76RYFT632599891...,2022-04-17,18:41:46,87.249.31.167,22f4:bb55:bc34:9a...,37237-974,031940993,1417054,031940993,03.854.176/0001-07,2018-01-06,8713441956769,1-380-31131-4,798,16.98


In [14]:
# Fazendo cross join entre os datasets autores e livros (tudo com tudo)
autores.join(livros, how='cross').orderBy('titulo')

id,titulo,autor,id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
106042959,1984,George Orwell,080608956,90.563.247/0001-09,2011-10-05,0332981484891,1-925760-25-1,203,107.63
106042959,1984,George Orwell,104361784,46.527.081/0001-83,2013-12-08,1491898510198,1-101-80944-2,478,109.14
106042959,1984,George Orwell,018453254,95.367.280/0001-87,2012-11-02,0507454098268,0-609-21722-4,043,52.94
106042959,1984,George Orwell,064175052,65.301.927/0001-07,2008-05-29,6905480714673,1-876722-64-9,795,85.56
106042959,1984,George Orwell,021765997,59.374.801/0001-60,2014-05-17,4743679337089,1-85580-232-5,499,143.43
106042959,1984,George Orwell,088482271,16.249.378/0001-63,2022-09-07,5713629047213,1-229-68842-0,632,143.95
106042959,1984,George Orwell,023911383,98.526.047/0001-98,2005-11-12,4165252098109,1-369-22273-4,599,121.0
106042959,1984,George Orwell,043636240,16.938.042/0001-08,2019-05-11,1239569754256,0-611-23680-X,371,207.67
106042959,1984,George Orwell,029667220,20.136.498/0001-21,2008-09-22,9813450961422,0-466-92503-4,474,175.32
106042959,1984,George Orwell,049806867,93.617.240/0001-10,2022-02-17,3166123637398,1-4822-8784-6,378,162.97


In [15]:
# Fazendo anti join entre os datasets autores e livros (somente os exclusívos de autores)
autores.join(livros, 'id', 'anti')

id,titulo,autor
106042959,1984,George Orwell
054099410,A Alma Encantador...,João do Rio
128422117,A Biblioteca da M...,Matt Haig
049141399,A Coleira do Cão,Rubem Fonseca
065490862,A Cor Púrpura,Alice Walker
029077001,A Educação Sentim...,Gustave Flaubert
090690941,A Estrela Sobe,Marques Rebelo
036365230,A Hora dos Rumina...,José J. Veiga
069146824,A Letra Escarlate,Nathaniel Hawthorne
076938216,A Moreninha,Joaquim Manuel de...


In [16]:
# Fazendo anti join entre os datasets livros e autores (somente os exclusívos de livros)
livros.join(autores, 'id', 'anti')

id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
102961160,54.317.982/0001-79,2011-09-19,4367115749184,0-06-594558-1,516,113.84
043636240,16.938.042/0001-08,2019-05-11,1239569754256,0-611-23680-X,371,207.67
052928059,57.624.038/0001-52,2003-08-03,5462162528221,1-109-29542-1,198,107.89
088482271,16.249.378/0001-63,2022-09-07,5713629047213,1-229-68842-0,632,143.95
028125446,90.425.178/0001-77,2011-07-25,2226757040245,0-657-66391-3,384,156.49
055610005,20.756.194/0001-67,2006-03-22,2114468195241,0-455-43278-3,419,37.08
049806867,93.617.240/0001-10,2022-02-17,3166123637398,1-4822-8784-6,378,162.97
102803370,48.760.531/0001-09,2011-12-13,2550608407612,0-399-05121-X,732,75.62
084858867,10.752.843/0001-06,2015-04-30,6207011925788,1-03-549652-6,635,237.67
059303729,05.129.783/0001-21,2001-08-25,1641133648784,0-620-46508-5,107,61.41


In [22]:
# Fazendo semi join entre os datasets compras e livros 
# semlhante ao inner, só que nesse caso é retirado as duplicatas, se houver
compras.join(livros, [compras.cd_livro == livros.id], 'semi')

id,cartao_data_expiracao,cartao_numero,cartao_bandeira,cartao_cvc,codigo_transacao_bancaria,data,hora,ipv4,ipv6,cep_entrega,cd_livro,cd_cliente
012389,11/25,5500804500517692,Discover,959,GB98MPIH622108593...,2021-07-24,03:21:28,62.145.31.164,b23d:58a2:9eff:36...,36629-219,030334762,3339828
012476,01/31,4609489235873,VISA 16 digit,6979,GB79GAVL233010819...,2021-08-23,11:15:52,185.150.224.52,1e2c:f641:49d1:5a...,17012-747,013721981,7624624
012478,07/23,30072722359174,American Express,689,GB51SZOU538484531...,2020-01-27,17:08:10,40.179.153.24,9bcd:ee0c:af68:33...,25921298,019458805,8703114
012534,09/29,4561935154572,VISA 16 digit,353,GB05UNEX021466511...,2021-07-15,07:39:25,86.20.51.194,be57:8f94:132:1cb...,73799370,014347542,7799936
012549,01/31,3541220668415122,JCB 15 digit,967,GB97RCCC581942620...,2020-07-22,23:56:14,55.189.220.65,a1fd:9dd:115f:9d5...,50727-454,020215846,6703678
012574,06/24,4155061214506542,JCB 16 digit,362,GB93YATI197164292...,2020-05-19,10:47:09,212.26.253.42,1815:dc0e:c557:96...,67662-182,010325500,6273720
012579,01/27,30554213514227,Maestro,9847,GB54PQLN799705859...,2021-04-15,10:36:56,168.197.230.167,7f3b:4ada:4c62:3f...,61193-110,053479015,6977964
012648,05/24,347386847428278,VISA 16 digit,823,GB90MYZL510193283...,2021-06-19,13:07:48,146.108.131.11,c52d:53d8:371a:34...,10059-723,012331534,6616715
012675,04/29,4534805013764,Mastercard,384,GB11XMUH824876351...,2021-07-11,07:55:08,148.155.72.234,b290:d2b7:6a58:86...,79292884,035940339,649001
012698,12/31,4450657280386776,American Express,247,GB79ECRO017868447...,2021-08-11,12:06:48,190.135.5.172,7cbc:4554:7f5d:65...,93609312,058320651,4523531
